In [ ]:
import random 
import time
import math
from IPython.display import clear_output
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 
            'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}
numbered_hands_list = {0:'first hand',1:'second hand',2:'third hand',3:'fourth hand'}

class IntegerOutOfRange(Exception):
    pass


In [ ]:
class Card():
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
        
    def __str__(self):
        return f'{self.rank} of {self.suit}'

In [ ]:
class Deck:
    
    def __init__(self):
        # Note this only happens once upon creation of a new Deck
        self.all_cards = [] 
        for suit in suits:
            for rank in ranks:
                # This assumes the Card class has already been defined!
                self.all_cards.append(Card(suit,rank))
                
    def shuffle(self):
        # Note this doesn't return anything
        random.shuffle(self.all_cards)
        
    def deal_one(self):
        # Note we remove one card from the list of all_cards
        return self.all_cards.pop() 

In [ ]:
class Hand:
    
    def __init__(self,):

        self.cards_in_hand = [] 
        self.score = 0
        self.number_0f_Aces = 0
        self.second_hand = False
        self.spitable = True
        self.bet = 0
        self.lost = False
    
    def add_bet(self,money):
        self.bet += money
    
    def print_hand(self):
        if not self.cards_in_hand:
            print('no cards at hand')
            print(f'score: {self.score}')
        else:
            for card in self.cards_in_hand:
                print(f'{card.__str__()}')
                
            print(f'score: {self.score}')
            print('')
            
    def remove_one(self):
        return self.cards_in_hand.pop() 

    def add_one(self,new_card,player):
        self.cards_in_hand.append(new_card) 
        if new_card.rank == 'Ace':
            self.number_0f_Aces += 1
        self.score += new_card.value
        if self.score > 21 and self.number_0f_Aces > 0:
            self.score -= 10
            self.number_0f_Aces -= 1
        print(f'{player.name} draws a card...')
    
    def check_for_blackjack(self):
        return self.score == 21
    
    def is_splitable(self):
        if (self.spitable == True) and (len(self.cards_in_hand) == 2) and (self.cards_in_hand[0].value == self.cards_in_hand[1].value):
            return True
        else: 
            return False
 
    def split(self,player):
        if self.second_hand == False:
            self.second_hand = True
        else:
            self.spitable = False
        
        for index,hand in enumerate(player.hands):
           
            if hand is self:
                new_hand = Hand()
                player.hands.insert(index+1, new_hand)
                del new_hand
                removed_card = self.remove_one()
                player.hands[index+1].add_one(removed_card,player)
                player.hands[index+1].bet = self.bet
                self.score -= removed_card.value 
                del removed_card
                break
                
            


In [ ]:
class dealers_hand(Hand):
    def __init__(self):
        Hand.__init__(self)
        self.hidden_score = 0
        self.hidden_Aces = 0
    
    def add_one(self,new_card):
        self.cards_in_hand.append(new_card) 
        if new_card.rank == 'Ace':
            self.number_0f_Aces += 1
        self.score += new_card.value
        if self.score > 21 and self.number_0f_Aces > 0:
            self.score -= 10
            self.number_0f_Aces -= 1
        print('Dealer draws a card...')
    
    def calculate_hidden_score(self):
        for card in self.cards_in_hand[1:]:
            if card.rank == 'Ace':
                self.hidden_Aces += 1
            self.hidden_score += card.value
            if self.hidden_score > 21 and self.hidden_Aces > 0:
                self.hidden_score -= 10
                self.hidden_Aces -= 1    
        return self.hidden_score
   
    def print_dealers_hand(self):
        if not self.cards_in_hand:
            print('no cards at hand')
            print(f'score: {self.score}')
        else:
            print('hidden card')
            for card in self.cards_in_hand[1:]:
                print(f'{card.__str__()}')
                
            print(f'current score: {self.calculate_hidden_score()}')
            print('')

In [ ]:
class player():
    #def __init__(self,name,starting_money,goal):
    def __init__(self,name):
        self.name = name
        self.money = 0
        self.goal = 0
        self.hands=[]
        
    def goal_reached(self):
        return self.money >= self.goal
    
    def add_money(self,cash):
        self.money += cash
        
    def remove_money(self,cash):
        self.money -= cash
        

In [ ]:
def replay():
    
    want_to_replay = "wrong"
    acceplable_answers = ['Y','N']
    
    while want_to_replay not in acceplable_answers:
        want_to_replay = input('Do you want to replay? (Y or N):')
        print('')
        
        if want_to_replay not in acceplable_answers:
            print('Please choose between Y or N')
    
        
    return want_to_replay == 'Y'

In [ ]:
def del_EVERYTHING(mydeck,player,dhand):
    del mydeck.all_cards[:]
    del mydeck
    for hand in player.hands:
        del hand.cards_in_hand[:]
    del player.hands[:]
    del dhand.cards_in_hand[:]
    del dhand

In [ ]:
def player_hand_actions(hand,player,hand_counter):  
    while True:
        possible_actions = ['Hit','Stand']
        possible_actions_number = 2

        #formatong player's menu
        if  player.money - bet >= bet:
                possible_actions.append('Double down')
                possible_actions_number += 1
        else:
            possible_actions.append('You can not afford to double down!')

        if player.hands[hand].is_splitable():
            if player.money - bet >= bet:
                possible_actions.append('Split')
                possible_actions_number += 1
            else:
                possible_actions.append('You can not afford to split!')

        while True:
            if not hand_counter == 1:
                print(f"{player.name}'s {numbered_hands_list[hand]}:")
            print(f"Bet:{player.hands[hand].bet}")
            player.hands[hand].print_hand()
            print(f'Choose your next move (1-{possible_actions_number})')
            
            #print player's menu
            for index,action in enumerate(possible_actions,1):
                if action == 'You can not afford to double down!':
                    print('You can not afford to double down!')

                elif action == 'You can not afford to split!':
                    print('You can not afford to split!')

                else:
                    print(f'{index}.{action}')

            try:
                player_choice = int(input('Your choice:'))
                if  player_choice < 0 or player_choice > possible_actions_number:
                        raise IntegerOutOfRange(f'Your choice must me an integer between 1 and {possible_actions_number},please try again.')
                print('')
            except IntegerOutOfRange as e:
                print(e)
                print('')
                continue
            except:
                print('Incorrect input, please try again')
                print('')
                continue
            else:
                player_choice -= 1
                break
        player_action = possible_actions[player_choice]
        print(f'You chose to {player_action.lower()}')
        print('')
        

        if player_action == 'Hit':
            player.hands[hand].add_one(mydeck.deal_one(),player)

            #checking for player's bust
            if player.hands[hand].score > 21:
                print(f'{player.name} busts.')
                player.hands[hand].print_hand()
                print(f'{player.name} lost {player.hands[hand].bet}$.')
                print('')
                player.money -= player.hands[hand].bet
                player.hands[hand].lost = True
                break

        elif player_action == 'Stand':
            break
        elif player_action == 'Double down':
            player.hands[hand].add_one(mydeck.deal_one(),player)
            player.hands[hand].print_hand()

            #checking for player's bust
            if player.hands[hand].score > 21:
                print(f'{player.name} busts.')
                print(f'{player.name} lost {player.hands[hand].bet * 2}$.')
                print('')
                player.money -= player.hands[hand].bet * 2
                player.hands[hand].lost = True
                break
            player.hands[hand].bet = player.hands[hand].bet * 2
            break
        elif player_action == 'Split':
            player.hands[hand].split(player)
            hand_counter += 1
            break

    return hand_counter

In [ ]:
minimum_bet = 5
print(f'Welcome to a game of Blackjack with a minimum bet of {minimum_bet}$')
#player info
while True:
    try:
        name = input('What is your name?')
        if not name:
            raise ValueError('Empty string, please try again')
    except ValueError as e:
        print(e)
        continue
    else:
        break


player = player(name) 

#main game
while True:
    while True:
        try:
            starting_money = int(input('How much is your starting money?'))
            if  starting_money < minimum_bet:
                    raise IntegerOutOfRange(f'Starting money must be more than or equal to the minimum bet of {minimum_bet}$.')
            print('')
        except IntegerOutOfRange as e:
            print(e)
            print('')
            continue
        except:
            print('Incorrect money input, please try again')
            print('')
            continue
        else:
            player.money = starting_money
            break

    while True:
        try:
            goal = int(input('How much is your goal money?'))
            if  starting_money >= goal:
                raise IntegerOutOfRange('Goal money must be more than starting money.')
            print('')
        except IntegerOutOfRange as e:
            print(e)
            print('')
            continue
        except:
            print('Incorrect goal money input, please try again')
            print('')
            continue
        else:
            player.goal = goal
            break

    clear_output()
    game_round = 0
    
    while True:
        #winning or losing conditions
        if player.money < minimum_bet:
            print('Player lost the game!')
            print(f'{player.name} has {player.money}$ and the minimum bet is {minimum_bet}$')
            print('')
            break
        elif player.goal_reached():
            print('Player Won the game!')
            print(f'{player.name} has {player.money}$ and the goal was {player.goal}$')
            print('')
            break
        
        #round start
        clear_output()
        game_round += 1
        print(f'Round: {game_round}')
        mydeck = Deck()
        mydeck.shuffle()
        dhand = dealers_hand()
        player.hands.append(Hand())
        print(f'{player.name} has {player.money}$')
        #placing bet
        while True:
            try:
                bet = int(input('Your bet?:'))
                if  bet < minimum_bet:
                        raise IntegerOutOfRange(f'Your bet must be more than or equal to the minimum bet of {minimum_bet}$.')
                if  bet > player.money:
                        raise IntegerOutOfRange(f'Your bet can not be more than your money({player.money}$)')
                print('')
            except IntegerOutOfRange as e:
                print(e)
                print('')
                continue
            except:
                print('Incorrect money input, please try again')
                print('')
                continue
            else:
                player.hands[0].bet = bet
                break

        #dealing initial cards
        player.hands[0].add_one(mydeck.deal_one(),player)
        player.hands[0].print_hand()
        time.sleep(1)
        dhand.add_one(mydeck.deal_one())
        dhand.print_dealers_hand()
        time.sleep(1)
        player.hands[0].add_one(mydeck.deal_one(),player)
        player.hands[0].print_hand()
        time.sleep(1)
        dhand.add_one(mydeck.deal_one())
        dhand.print_dealers_hand()
        time.sleep(1)
        
        #checking for player's blackjack
        if player.hands[0].check_for_blackjack():
            print(f'{player.name} has blackjack!')
            print(f'{player.name} won this round!')
            print(f'{player.name} won {round(player.hands[0].bet/2)}$')
            print('')
            player.money += round(player.hands[0].bet/2)
                             
            del_EVERYTHING(mydeck,player,dhand)
            time.sleep(5)
            continue
        
        #insurance 
        want_insurance_bet = 'wrong'
        if dhand.cards_in_hand[1].rank == 'Ace':
            while True:
                try:
                    want_insurance_bet = input('Do you want to place an insurance bet? (Y or N):')
                    print('')
                    if  want_insurance_bet not in ['Y','N']:
                            raise Error()
                except:
                    print('Please choose between Y or N')
                    continue
                else:
                    break
            if want_insurance_bet == 'Y':
                print(f'Half of your bet is:{float(player.hands[0].bet/2)}')
                while True:
                    try:
                        insurance_bet = float(input('What is your insurance bet? (up to half your original bet and can be float):'))
                        if  insurance_bet > float(player.hands[0].bet/2):
                                raise IntegerOutOfRange('Your insurance bet must be up to half your original bet, please try again.')
                        if  insurance_bet <= 0:
                                raise IntegerOutOfRange('Your insurance bet must be more than 0')
                        print('')
                    except IntegerOutOfRange as e:
                        print (e)
                        print('')
                        continue
                    except:
                        print('Incorrect money input, please try again.')
                        print('')
                        continue
                    else:
                        break
        
        #checking for dealer's blackjack
        print('The dealer is checking for blakjack...')
        time.sleep(1)
        print('')
        if dhand.check_for_blackjack():
            print('The dealer has blakjack!')
            print('The dealer won this round!!')
            print('')
            print(f'{player.name} lost {player.hands[0].bet}$')
            player.money -= player.hands[0].bet
            if want_insurance_bet == 'Y':
                print(f'{player.name} won {int(round(insurance_bet * 2))}$ due to insurance bet')
                print('')
                player.money += int(round(insurance_bet * 2))   
            del_EVERYTHING(mydeck,player,dhand)
            time.sleep(5)
            continue
        else:
            print('The dealer does not have blakjack')
            print('')
            if want_insurance_bet == 'Y':
                print(f'{player.name} lost {insurance_bet}$ due to insurance bet')
                print('')
                player.money -= insurance_bet
        
        #surrender
        while True:
            try:
                surrender = input('Do you want to surrender this round? (Y or N):')
                print('')
                if  surrender not in ['Y','N']:
                        raise Error()
            except:
                print('Please choose between Y or N')
                print('')
                continue
            else:
                break 
        if surrender == 'Y':
            print(f'{player.name} lost {round(player.hands[0].bet/2)}$ due to surrendering')
            print('')
            player.money -= round(player.hands[0].bet/2)
            del_EVERYTHING(mydeck,player,dhand)
            time.sleep(5)
            continue
        
        #player's turn
        hand_counter = 1
        hand_num = 0
        while hand_num < hand_counter:

            hand_counter_new = player_hand_actions(hand_num,player,hand_counter)
            hand_num += 1
            if not hand_counter_new == hand_counter:
                hand_counter = hand_counter_new
                hand_num = 0
        
        #check if all hands lost
        lost_hands_count = 0
        for hand in player.hands:
            if hand.lost:
                lost_hands_count += 1
        if lost_hands_count == hand_counter:
            del_EVERYTHING(mydeck,player,dhand)
            time.sleep(5)
            continue   
        
        #dealer's turn
        print('Dealer reveals his hidden card...')
        time.sleep(1)
        while True: 
            dhand.print_hand()
            
            possible_losses = 0
            for hand in player.hands:
                if dhand.score > hand.score:
                    possible_losses += hand.bet
                elif dhand.score < hand.score:
                    possible_losses -= hand.bet
            
            if possible_losses < 0:
                #hit
                dhand.add_one(mydeck.deal_one())
                time.sleep(1)

                #checking for dealer's bust
                if dhand.score > 21:
                    print(f'Dealer busts.')
                    dhand.print_hand()
                    dhand.lost = True
                    
                    for index,hand in enumerate(player.hands):
                        
                        if not hand.lost:
                            if hand_counter > 1:
                                print(f'{player.name} won {hand.bet}$ due to {numbered_hands_list[index]}.')
                            else:
                                print(f'{player.name} won {hand.bet}$.')
                            print('')
                            player.money += hand.bet
                    break
            else:
                #stand
                break
        
        #final calculations
        if not dhand.lost:
            for index,hand in enumerate(player.hands):
                        
                if not hand.lost:
                    if hand.score > dhand.score:
                        if hand_counter > 1:
                            print(f'{player.name} won {hand.bet}$ due to {numbered_hands_list[index]}.')
                        else:
                            print(f'{player.name} won {hand.bet}$.')
                        print('')
                        player.money += hand.bet
                    elif hand.score < dhand.score:
                        if hand_counter > 1:
                            print(f'{player.name} lost {hand.bet}$ due to {numbered_hands_list[index]}.')
                        else:
                            print(f'{player.name} lost {hand.bet}$.')
                        print('')
                        player.money -= hand.bet
                    else:
                        if hand_counter > 1:
                            print(f"{player.name}'s {numbered_hands_list[index]} ties with Dealer.")
                        else:
                            print(f'{player.name} ties with Dealer.')
                        print('')
                 
        del_EVERYTHING(mydeck,player,dhand)
        time.sleep(5)
    
    if not replay():
        break